In [ ]:
#Packages
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

#Load dataset
file = r"C:\Users\vavoo\OneDrive\Desktop\Online_Retail5.csv"
data = pd.read_csv(file, sep=';', encoding='latin1')
print(data.head())

#######################################################################################################

# 1. preprocessing and cleaning data

#Exploring dataset
print(data.info())
print(data.columns)
data = data.rename(columns={'Country,,,': 'Country'})
print(data.describe())

#Visualizing missing values
sns.heatmap(data.isnull(), yticklabels=False, cbar=False, cmap='viridis')
plt.show()
print(data.isnull().sum())

#Removing missing values
clean_data = data.dropna(subset=['Quantity', 'InvoiceDate', 'UnitPrice', 'Country'])
clean_data = clean_data.dropna(subset=['Description'])
clean_data = clean_data.dropna(subset=['CustomerID'])
#Check
print(clean_data.isnull().sum())
print(clean_data.info())

#Clean UnitPrice column
clean_data['UnitPrice'] = clean_data['UnitPrice'].str.replace(",", ".")
clean_data['UnitPrice'] = clean_data['UnitPrice'].astype('float64')
clean_data = clean_data[clean_data['UnitPrice'] > 0.00]

#Clean Quantity column
clean_data['Quantity'] = clean_data['Quantity'].astype('int64')
negative_quantity = clean_data[clean_data['Quantity'] <= 0]
print(negative_quantity.value_counts())
clean_data = clean_data[clean_data['Quantity'] > 0]

#Clean StockCode column
clean_data['StockCode'] = clean_data['StockCode'].astype('string')
clean_data = clean_data[~clean_data['StockCode'].isin(['M', 'B', 'C2', 'AMAZONFEE', 'D', 'DOT', 'POST', 'BANK CHARGES', 'CRUK', 'S'])]

#Clean Description Column
clean_data['Description'] = clean_data['Description'].astype('string')
clean_data['Description'] = clean_data['Description'].str.upper()
print(clean_data[clean_data['Description'] == 'ADJUST BAD DEBT'])

#Clean CustomerID column
clean_data['CustomerID'] = clean_data['CustomerID'].astype('string')
print(clean_data['CustomerID'].nunique())

#Clean InvoiceNo column
clean_data['InvoiceNo'] = clean_data['InvoiceNo'].astype('string')

#Clean InvoiceDate column and extracting day, month and year
clean_data['InvoiceDate'] = pd.to_datetime(clean_data['InvoiceDate'], format='%d-%m-%Y %H:%M', errors='coerce')
clean_data['Year'] = clean_data['InvoiceDate'].dt.year
clean_data['Month'] = clean_data['InvoiceDate'].dt.month
clean_data['DayOfWeek'] = clean_data['InvoiceDate'].dt.day_name()
print(clean_data[['InvoiceDate', 'Year', 'Month', 'DayOfWeek']].head())

#Clean Country column
clean_data['Country'] = clean_data['Country'].astype('category')
clean_data['Country'] = clean_data['Country'].str.replace(",", "")
clean_data['Country'] = clean_data['Country'].str.replace("EIRE", "Ireland")
clean_data['Country'] = clean_data['Country'].str.replace("RSA", "South Africa")
countries_unique = list(clean_data['Country'].unique())
print(countries_unique)
print(clean_data['Country'].value_counts())

#create clean_data csv
clean_data.to_csv('clean_data.csv', index=False)

######################################################################################################

# 2. mapping and adding columns

clean_data = pd.read_csv('clean_data.csv')

#Mapping countries to continents
mappings = {'United Kingdom':'Europe', 'Germany':'Europe', 'France': 'Europe',
            'Ireland': 'Europe', 'Spain': 'Europe',
            'Netherlands': 'Europe', 'Belgium': 'Europe', 'Switzerland': 'Europe', 'Portugal': 'Europe',
            'Australia': 'Oceania', 'Norway': 'Europe', 'Channel Islands': 'Europe', 'Italy': 'Europe',
            'Finland': 'Europe', 'Cyprus': 'Europe', 'Sweden': 'Europe', 'Austria': 'Europe',
            'Denmark': 'Europe', 'Poland': 'Europe', 'Japan': 'Asia', 'Israel': 'Middle East',
            'Unspecified': 'Unspecified', 'Singapore': 'Asia', 'Iceland': 'Europe', 'USA': 'North America',
            'Canada': 'North America', 'Greece': 'Europe', 'Malta': 'Europe',
            'United Arab Emirates': 'Middle East', 'European Community': 'Europe', 'South Africa': 'Africa',
            'Lebanon': 'Middle East', 'Lithuania': 'Europe', 'Brazil': 'South America',
            'Czech Republic': 'Europe', 'Bahrain': 'Middle East', 'Saudi Arabia': 'Middle East'}
clean_data['Continent'] = clean_data['Country'].replace(mappings)
print(clean_data[['Country', 'Continent']].head())

#Adding TotalSales column
clean_data['TotalSales'] = clean_data['Quantity'] * clean_data['UnitPrice']
print(clean_data[['Quantity', 'UnitPrice', 'TotalSales']].head())
print(clean_data['TotalSales'].describe())

#Adding handling fees to WsFee column
clean_data['WsFee'] = np.where(
    clean_data['TotalSales'] <= 100000,
    clean_data['TotalSales'] * 0.0025,
    np.where(
        clean_data['TotalSales'] <= 1000000,
        clean_data['TotalSales'] * 0.0019,
        clean_data['TotalSales'] * 0.0016))

print(clean_data[['TotalSales', 'WsFee']].head())

#create new csv
clean_data.to_csv('clean_data2.csv', index=False)

####################################################################################################

# 3. exploratory data analysis plus visualizations

#Load and create new csv
sales = pd.read_csv('clean_data2.csv')
sales.to_csv('sales.csv', index=False)

#Exploraring outliers

sns.boxplot(x=sales['Quantity'])
sns.boxplot(x=sales['UnitPrice'])
plt.show()

# Total amount of handling fees earned by WireSolution

total_fees = sales['WsFee'].sum().round()
print(f"total handling fees amount to: ", total_fees)

#Exploring total sales per country

total_sales_countries = sales.groupby('Country')['TotalSales'].sum()
top_5_countries = total_sales_countries.nlargest(5)
print(top_5_countries)
# Scale sales to thousands
top_5_countries_in_thousands = top_5_countries / 1000
# Create the bar plot
top_5_countries_in_thousands.plot(kind='bar', color='skyblue')
# Add labels and title
plt.xlabel('Country')
plt.ylabel('Total Sales (in Thousands)')
plt.title('Top 5 Countries by Total Sales')
# Rotate x-axis labels for readability
plt.xticks(rotation=45)
plt.show()

#Exploring best sold products

best_sold_products = sales.groupby(['StockCode', 'Description'])['Quantity'].sum()
top_5_products = best_sold_products.nlargest(5)
print(top_5_products)
# create bar plot
top_5_products.plot(kind='bar', color='skyblue')
# Add labels and title
plt.xlabel('Product')
plt.ylabel('Quantity Sold')
plt.title('Top 5 Best-Selling Products')
# Rotate x-axis labels for readability
plt.xticks(rotation=45)
plt.show()

#Exploring monthly transactions

monthly_transactions = sales.groupby('Month')['InvoiceNo'].count().reset_index()
print(monthly_transactions)
sns.lineplot(data=monthly_transactions, x='Month', y='InvoiceNo', marker='o')
# Add labels and title
plt.xlabel('Months')
plt.ylabel('Transaction Count')
plt.title('Monthly Transactions Line Plot')
# Rotate x-axis labels for better readability
plt.xticks(rotation=45)
plt.show()

#Exploring daily transactions
daily_transactions = sales.groupby('DayOfWeek')['InvoiceNo'].count().reset_index()
daily_transactions.columns = ['DayOfWeek', 'TransactionCount']
days_order = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']
daily_transactions['DayOfWeek'] = pd.Categorical(daily_transactions['DayOfWeek'], categories=days_order, ordered=True)
daily_transactions = daily_transactions.sort_values('DayOfWeek')

# Create the line plot
sns.lineplot(data=daily_transactions, x='DayOfWeek', y='TransactionCount', marker='o')
# Add labels and title
plt.xlabel('Day of Week')
plt.ylabel('Transaction Count')
plt.title('Daily Transactions Line Plot')
# Rotate x-axis labels for better readability
plt.xticks(rotation=45)
# Display the plot
plt.show()

#Exploring hourly transactions

sales['InvoiceDate'] = pd.to_datetime(sales['InvoiceDate'])
sales['Hour'] = sales['InvoiceDate'].dt.hour
# Count transactions per hour
hourly_transactions = sales.groupby('Hour')['InvoiceNo'].count().reset_index()
hourly_transactions.columns = ['Hour', 'TransactionCount']
# Line plot hourly transactions
sns.lineplot(data=hourly_transactions, x='Hour', y='TransactionCount', marker='o')
plt.xlabel('Hour of Day')
plt.ylabel('Transaction Count')
plt.title('Hourly Transactions Line Plot')
plt.xticks(hourly_transactions['Hour'])
plt.show()

#create new csv
sales.to_csv('rfm.csv', index=False)

###################################################################################################

# 4. rfm modelling: customer segmentation based on three variables

#Load and create new csv
rfm = pd.read_csv('rfm.csv')

# Recency: number of days since last purchase
rfm['InvoiceDate'] = pd.to_datetime(rfm['InvoiceDate'])
rfm['Date'] = pd.DatetimeIndex(rfm['InvoiceDate']).date
print(rfm[['InvoiceDate', 'Date']].head())
# create variable latest transaction as 'now'
now = rfm['Date'].max()
print(now)
# create DataFrame with latest purchase date of each unique customer
recency_df = rfm.groupby('CustomerID')['Date'].max().reset_index()
recency_df['Recency'] = recency_df['Date'].apply(lambda x: (now - x).days)
# check
print(recency_df.head())
# drop date column
recency_df.drop('Date',axis=1,inplace=True)
print(recency_df.head())

# Frequency: total number of transaction by each unique customer
frequency_df = rfm.groupby('CustomerID')['InvoiceNo'].count().reset_index()
frequency_df.columns = ['CustomerID','Frequency']
print(frequency_df.head())

# Monetary: total value of transactions by each unique customer
monetary_df = rfm.groupby('CustomerID')['TotalSales'].sum().reset_index()
monetary_df.columns = ['CustomerID','Monetary']
print(monetary_df.head())

# Merge all three dataframes
merged_df = recency_df.merge(frequency_df, on='CustomerID', how='inner').merge(monetary_df, on='CustomerID', how='inner')
# Set customerid as index
merged_df.set_index('CustomerID',inplace=True)
print(merged_df.head())
print(merged_df.describe())

# in order to make customer segments, a score needs to be assigned to these three variables
# we score between 1 and 4 with 1 being the worst and 4 being the best
# note that a high value of recency is bad, while a high value of frequency and monetary is good

# to score 1 through 4 we need to use quartiles
quantiles = merged_df.quantile(q=[0.25,0.5,0.75])
quantiles = quantiles.to_dict()
print(quantiles)

# First we create a function to score recency (RScore), then a function to score both frequency and monetary (FMScore)


# Arguments (x = value, p = recency, frequency, monetary d = quantiles dict)
def RScore(x,p,d):
    if x <= d[p][0.25]:
        return 4
    elif x <= d[p][0.50]:
        return 3
    elif x <= d[p][0.75]:
        return 2
    else:
        return 1

# Arguments (x = value, p = recency, frequency, monetary k = quantiles dict)
def FMScore(x,p,d):
    if x <= d[p][0.25]:
        return 1
    elif x <= d[p][0.50]:
        return 2
    elif x <= d[p][0.75]:
        return 3
    else:
        return 4

# create rfm segmentation table
rfm_segmentation = merged_df
rfm_segmentation['R_Score'] = rfm_segmentation['Recency'].apply(RScore, args=('Recency',quantiles,))
rfm_segmentation['F_Score'] = rfm_segmentation['Frequency'].apply(FMScore, args=('Frequency',quantiles,))
rfm_segmentation['M_Score'] = rfm_segmentation['Monetary'].apply(FMScore, args=('Monetary',quantiles,))
print(rfm_segmentation.head())

# calculate final RFM score by combining 3 separate scores
rfm_segmentation['RFMScore'] = rfm_segmentation['R_Score'].map(str) + rfm_segmentation['F_Score'].map(str) + rfm_segmentation['M_Score'].map(str)
print(rfm_segmentation['RFMScore'].head())

# maximum number of groups is 4**4 = 64
print(rfm_segmentation['RFMScore'].nunique())

# number of customers in different segments
print("Best Customers: ",len(rfm_segmentation[rfm_segmentation['RFMScore']=='444']))
print('Loyal Customers: ',len(rfm_segmentation[rfm_segmentation['F_Score']==4]))
print("Big Spenders: ",len(rfm_segmentation[rfm_segmentation['M_Score']==4]))
print('Almost Lost: ', len(rfm_segmentation[rfm_segmentation['RFMScore']=='244']))
print('Lost Customers: ',len(rfm_segmentation[rfm_segmentation['RFMScore']=='144']))
print('Lost Cheap Customers: ',len(rfm_segmentation[rfm_segmentation['RFMScore']=='111']))

# visualizing customer segments
# Data for the segments
segments = ['Best Customers', 'Loyal Customers', 'Big Spenders',
            'Almost Lost', 'Lost Customers', 'Lost Cheap Customers']
counts = [
    len(rfm_segmentation[rfm_segmentation['RFMScore'] == '444']),
    len(rfm_segmentation[rfm_segmentation['F_Score'] == 4]),
    len(rfm_segmentation[rfm_segmentation['M_Score'] == 4]),
    len(rfm_segmentation[rfm_segmentation['RFMScore'] == '244']),
    len(rfm_segmentation[rfm_segmentation['RFMScore'] == '144']),
    len(rfm_segmentation[rfm_segmentation['RFMScore'] == '111'])
]
# Create the bar plot
plt.bar(segments, counts, color='skyblue')
# Add labels and title
plt.xlabel('Customer Segments')
plt.ylabel('Number of Customers')
plt.title('Number of Customers in Each Segment')
# Rotate x-axis labels for readability
plt.xticks(rotation=45, ha='right')
plt.show()